In [1]:
import numpy as np

In [ ]:
# import pretrained model for transfer learning

from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
# import cv2


processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")


In [21]:
# import json reader
import json
import os
boxes = []
# open the file:
for f in os.listdir('TRAINING'):
    if f.endswith(".json"):
        with open('TRAINING/'+f) as f:
            data = json.load(f)
            try:
                boxes.append(data["shapes"][1]["points"])
            except:
                print("error", f)

error <_io.TextIOWrapper name='TRAINING/Capture10.json' mode='r' encoding='UTF-8'>
error <_io.TextIOWrapper name='TRAINING/Capture9.json' mode='r' encoding='UTF-8'>


In [6]:

url = "./TRAINING/Capture.PNG"
image = Image.open(url).convert("RGB")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.1)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )


Detected fire hydrant with confidence 0.621 at location [734.42, 3.95, 1110.76, 471.46]
Detected parking meter with confidence 0.145 at location [751.08, 0.44, 1116.11, 81.1]


In [8]:
results

{'scores': tensor([0.6213, 0.1453], grad_fn=<IndexBackward0>),
 'labels': tensor([11, 14]),
 'boxes': tensor([[7.3442e+02, 3.9522e+00, 1.1108e+03, 4.7146e+02],
         [7.5108e+02, 4.4197e-01, 1.1161e+03, 8.1104e+01]],
        grad_fn=<IndexBackward0>)}